# 2 - Hough Transform aplicado a puntos Discretos en una Esfera

Generar puntos que dibujen una circunferencia en la superficie de una esfera y despues intentar aplicar la transformada de Hough de modo a recuperar los parametros de esas circunferência.

In [1]:
import uproot

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn as sn
from tqdm import tqdm
from matplotlib import style

import numpy as np
from numpy import array, where, shape, reshape, pi, cos, sin, sqrt, linspace
np.set_printoptions(threshold=1000)

from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter

In [2]:
file = uproot.open("/snoplus simulations/py_out1.root")

## 2.1 PMT data base is used as points

In [3]:
pmt_info = file['pmt;1']
pmt_info.keys()

['pmt_id', 'pmt_pos_xyz', 'pmt_pos_sph', 'pmt_type']

In [4]:
pmt_id = array(pmt_info['pmt_id'])
pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
pmt_type = array(pmt_info['pmt_type'])

In [5]:
pmt_pos_sph

array([[ 2.18627604e+00, -2.35619449e+00,  1.73203349e+05],
       [ 2.43505602e+00,  1.16025037e-02,  8.42248856e+03],
       [ 2.40917419e+00,  4.74785216e-02,  8.40698104e+03],
       ...,
       [ 6.54226614e-01, -1.48191511e+00,  9.20079013e+03],
       [ 7.32698456e-01, -8.70981952e-01,  9.20131295e+03],
       [ 3.22890171e-01, -8.01377433e-01,  9.20467647e+03]])

In [6]:
#Lets separate their coordinates
#xyz:

prefix_xyz = 'type_'
suffix_xyz = '_xyz'

type_var = np.unique(pmt_type)

for i in type_var:
    condition = (pmt_type == i)
    locals()[prefix_xyz + str(i) + suffix_xyz] = []
    
    for j in (where(condition)[0]):
        locals()[prefix_xyz + str(i) + suffix_xyz].append(pmt_pos_xyz[j])
    locals()[prefix_xyz + str(i) + suffix_xyz] = np.array(locals()[prefix_xyz + str(i) + suffix_xyz])
    
#spherical:

prefix_sph = 'type_'
suffix_sph = '_sph'

type_var = np.unique(pmt_type)

for i in type_var:
    condition = (pmt_type == i)
    locals()[prefix_sph + str(i) + suffix_sph] = []
    
    for j in (where(condition)[0]):
        locals()[prefix_sph + str(i) + suffix_sph].append(pmt_pos_sph[j])
    locals()[prefix_sph + str(i) + suffix_sph] = np.array(locals()[prefix_sph + str(i) + suffix_sph])

In [7]:
# Choice of PMT 'ring' -> usar grosura de angulo zenital para seleccionar conjuntos de PMTs

delta_alpha = 5 #degrees
delta_rad = pi*delta_alpha/180 #pass to radians

#choose zenith
zenit_1 = 3*pi/4
zenit_2 = zenit_1 + delta_rad

#choose azimuth
azimut_1 = -pi/2 
azimut_2 =  pi/2

sector_xyz = []

size = shape(type_1_sph)[0]

for i in range(size):
    
    #cos_azimut = cos(type_1_sph[i][0])
    #cos_polar = cos(type_1_sph[i][1])
    zenit = type_1_sph[i][0]
    azimut = type_1_sph[i][1]
    
    #choose the sector
    if (zenit >= zenit_1 and zenit <= zenit_2):
        sector_xyz.append(type_1_xyz[i])
        
sector_xyz = np.array(sector_xyz)

In [9]:
%matplotlib
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()


x = sector_xyz[:,0]
y = sector_xyz[:,1]
z = sector_xyz[:,2]

ax.scatter(x, y, z, c = 'r', s = 5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.legend()

ax.axes.set_xlim3d(left=-8000, right=8000) 
ax.axes.set_ylim3d(bottom=-8000, top=8000) 
ax.axes.set_zlim3d(bottom=-8000, top=8000)

Using matplotlib backend: TkAgg


(-8000.0, 8000.0)

### Now, rotate the data by a rotation matrix $R_x(\theta)$

In [10]:
# Rotations matrix around x axis

theta = pi*45/180 #45 degree rotation to radian
R_x = np.array([[1, 0, 0], 
              [0, cos(theta), -sin(theta)], 
              [ 0, sin(theta), cos(theta)]])

rot_data_xyz = []

size = sector_xyz.shape[0]

for i in range(size):
    point = sector_xyz[i,:]
    rot_point = np.dot(point, R_x)
    rot_data_xyz.append(rot_point)
    
rot_data_xyz = np.array(rot_data_xyz)

### 2.3 Representation in xyz coordinates

In [11]:
sn.set_style(rc = {'axes.facecolor': 'white'})
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()

x_rot = rot_data_xyz[:,0]
y_rot = rot_data_xyz[:,1]
z_rot = rot_data_xyz[:,2]

ax.scatter(x_rot, y_rot, z_rot, c = 'r', s = 5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.legend()

ax.axes.set_xlim3d(left=-8000, right=8000) 
ax.axes.set_ylim3d(bottom=-8000, top=8000) 
ax.axes.set_zlim3d(bottom=-8000, top=8000)

(-8000.0, 8000.0)

In [12]:
title = 'xy plane'
bins = len(x_rot)

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = x_rot, y = y_rot, bins = [bins,bins], stat='count', cbar = 'True', cmap = cm.nipy_spectral)
plt.ylabel('y_pmt')
plt.xlabel('x_pmt')
# plt.xlim(-8000,8000)
# plt.ylim(-8000,8000)
plt.title(title)

#equal acis ration
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.show()

### 2.4 Representation in spherical coordinates

#### Function to pass from Cart. to Sphe.

In [13]:
def cart2spher(point_xyz):
    
    #point_xyz -> array with nx3 entries!
    
    import math
    
    x = point_xyz[:,0]
    y = point_xyz[:,1]
    z = point_xyz[:,2]
    

    # Calcular la distancia radial
    r = np.sqrt(x**2 + y**2 + z**2)
    
    # Calcular la latitud (theta)
    zenit = np.arccos(z / r)
    
    # Calcular la longitud (phi)
    azimut = np.arctan2(y,x) #arctan(y/x)
    
    return np.transpose(np.array([r, azimut, zenit]))


In [14]:
r = cart2spher(rot_data_xyz)[:,0]
azimut_rot = cart2spher(rot_data_xyz)[:,1]
zenit_rot = cart2spher(rot_data_xyz)[:,2]

In [15]:
title = '$\phi (\Theta))$ plane'
bins = 40

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = (zenit_rot), y = azimut_rot, bins = [bins,bins], stat='count', cmap = cm.nipy_spectral)
plt.ylabel('$azimuth$')
plt.xlabel('$zenith$')
# plt.xlim(-8000,8000)
# plt.ylim(-8000,8000)
plt.title(title)

#equal acis ration
# ax = plt.gca()
# ax.set_aspect('equal', adjustable='box')
plt.show()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\joanc\AppData\Local\Temp\ipykernel_23472\1323377076.py:1: SyntaxWarning: invalid escape sequence '\p'
  title = '$\phi (\Theta))$ plane'


In [526]:
title = '$\phi (cos(\Theta))$ plane'
bins = 40

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = np.cos(zenit_rot), y = azimut_rot, bins = [bins,bins], stat='count', cbar = 'True', cmap = cm.nipy_spectral)
plt.ylabel('$azimuth$')
plt.xlabel('$cos(zenith)$')
# plt.xlim(-8000,8000)
# plt.ylim(-8000,8000)
plt.title(title)

#equal acis ration
# ax = plt.gca()
# ax.set_aspect('equal', adjustable='box')
plt.show()

--------------------------------------------

#### Check if data is recovered -> YES

In [16]:
sphe = cart2spher(rot_data_xyz)[0,:]
x = sphe[0]*np.sin(sphe[2])*np.cos(sphe[1])
y = sphe[0]*np.sin(sphe[1])*np.sin(sphe[2])
z = sphe[0]*np.cos(sphe[2])

In [21]:
x 

5467.54

In [22]:
y

-4485.058104913468

In [23]:
z

-4574.775813310417

In [24]:
rot_data_xyz[0,:]

array([ 5467.54      , -4485.05810491, -4574.77581331])

-------------------------------------

## 2.5 Extraer pixeles

In [17]:
title = 'xy plane'
bins = 41

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
#imag contiene la imagen
imag_xyz, xbin, ybin, im = plt.hist2d(x = x_rot, y = y_rot, bins = [bins,bins],  cmap = cm.nipy_spectral) 
plt.ylabel('y_pmt')
plt.xlabel('x_pmt')
# plt.xlim(-8000,8000)
# plt.ylim(-8000,8000)
plt.title(title)

#equal acis ration
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.show()

In [18]:
plt.imshow(imag_xyz)

In [58]:
title = '$\phi (\Theta)$ plane'
bins = 50

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
imag_sphe , xbin, ybin, im = plt.hist2d(x = zenit_rot, y = azimut_rot, bins = [bins,bins],  cmap = cm.nipy_spectral) 
plt.ylabel('$azimuth$')
plt.xlabel('$zenith$')
# plt.xlim(-8000,8000)
# plt.ylim(-8000,8000)
plt.title(title)

#equal acis ration
# ax = plt.gca()
# ax.set_aspect('equal', adjustable='box')
plt.show()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\joanc\AppData\Local\Temp\ipykernel_23472\1171037903.py:1: SyntaxWarning: invalid escape sequence '\p'
  title = '$\phi (\Theta)$ plane'


In [21]:
plt.imshow(imag_sphe, aspect = 'auto')

## 2.5 Aplicar la transformada de Hough

### 2.5.1 xyz data - Identificar una elipse

In [182]:
len(x_rot)

280

In [256]:
bins = 50
imag_xyz, _, _ =  np.histogram2d(x = x_rot, y = y_rot, bins = [bins,bins])
plt.imshow(imag_xyz)
plt.show()

In [269]:
result_xyz = hough_ellipse(imag_xyz, accuracy = 1, threshold = 4)

In [270]:
shape(result_xyz)

(16051,)

In [271]:
type(result_xyz)

numpy.ndarray

In [272]:
result

array([( 5,  0.5, 22.5,  0.        ,  8.51469318, 3.08283683),
       ( 5,  0.5, 23.5,  0.        ,  7.51664819, 3.07502449),
       ( 5,  0.5, 26. ,  7.01783442,  0.        , 1.64210379), ...,
       (22, 24.5, 23.5, 23.50531855, 23.49468025, 2.23557082),
       (22, 24.5, 23.5, 23.50531855, 23.49468025, 2.47681816),
       (22, 24.5, 23.5, 23.50531855, 23.49468025, 2.72552815)],
      dtype=[('accumulator', '<i8'), ('yc', '<f8'), ('xc', '<f8'), ('a', '<f8'), ('b', '<f8'), ('orientation', '<f8')])

Esta estructura de np.array significa que por cada (data) los datos estan organizados como (accumulator, yc, xc, a, b, orientation)

In [273]:
# ordenar por acumulador creciente
result_xyz.sort(order = 'accumulator')

#mejor acumulador
best = list(result_xyz[-1])
best

[28, 24.5, 23.5, 23.50531854708632, 23.49468024894146, 3.120319267565732]

In [262]:
#extraer paramentos de elipse
yc, xc, a, b = (int(round(x)) for x in best[1:5]) #iterate over best[1:5]
orientation = best[5]

In [263]:
# extraer parametros de elipse
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
#cy, cx -> Index of pixels that belong to the ellipse perimeter

# Resaltar pixeles en la imagen original de modo a ver la elipse reconstruida
imag_xyz[cy,cx] = 5

In [264]:
#plt.title('Branco: Anel reconstruido | Azul: dados')
plt.imshow(imag_xyz, cmap = cm.nipy_spectral, aspect = 'auto')
plt.show()
#plt.savefig('figs/good recons/ellipse recons bins =' + str(bins) + '.jpg', format= 'jpg')

###  Observar máximos en el espacio de Hough usando xyz data

Existen maximos por fuera de la elipse o que no reconstruyen bien -> Observar estos máximos en una distribución 3D en el espacio de Hough

- Extraer campos de parametros

In [265]:
accumulator_field = result['accumulator']
xc_field = result_xyz['xc']
yc_field = result_xyz['yc']
a_field = result_xyz['a']
b_field = result_xyz['b']

- Observar su distribuciones 2D

In [266]:
title = 'Ellipse Center Fields - Hough Space'

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = a_field, y = b_field, bins = [50,50], stat='count', cbar = 'True', cmap = cm.nipy_spectral)
plt.xlabel('x center field')
plt.ylabel('y center field')
plt.title(title)

#equal acis ration
#ax = plt.gca()
#ax.set_aspect('equal', adjustable='box')
plt.show()
#plt.savefig('figs/good recons/(a,b) field bins =' + str(bins) + '.jpg', format= 'jpg')

In [267]:
title = 'Ellipse Axis Fields - Hough Space'

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = xc_field, y = yc_field, bins = [50,50], stat='count', cbar = 'True', cmap = cm.nipy_spectral)
plt.xlabel('x axis field')
plt.ylabel('y axis field')
plt.title(title)

#equal acis ration
#ax = plt.gca()
#ax.set_aspect('equal', adjustable='box')
plt.show()
#plt.savefig('figs/good recons/(xc,yc) field bins =' + str(bins) + '.jpg', format= 'jpg')

Como interpretar estos resultados?

Son los campos de los parametros a ajustar, y su maximo coincide con el mayor acumulador (interseción máxima)

- Observar histogramas a 3D

In [43]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

x, y = a_field, b_field
z = 0

dx = 1
dy = 1
dz = accumulator_field

style.use('seaborn-v0_8-colorblind')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#colors: https://matplotlib.org/stable/gallery/color/named_colors.html
ax.bar3d(x, y, z, dx, dy, dz, color='cornflowerblue' )
#ax.view_init(elev=24., azim=-33)

plt.title('Ellipse Center Fields - Hough Space')
ax.set_xlabel('x center field')
ax.set_ylabel('y center field')
ax.set_zlabel('Accumulator')
plt.show()

In [44]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

x, y = xc_field, yc_field
z = 0

dx = 1
dy = 1
dz = accumulator_field

style.use('seaborn-v0_8-colorblind')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#colors: https://matplotlib.org/stable/gallery/color/named_colors.html
ax.bar3d(x, y, z, dx, dy, dz, color='cornflowerblue' )
#ax.view_init(elev=24., azim=-33)

plt.title('Ellipse Axis Fields - Hough Space')
plt.xlabel('x axis field')
plt.ylabel('y axis field')
ax.set_zlabel('Accumulator')
plt.show()

Problemas de reconstrucciõn surgen al aumentar el número de pixeles en la imagen! ->
Probar coordenadas esfericas

### 2.5.2 spherical data - Identificar una elipse

In [304]:
bins = 55
imag_sph, _, _ =  np.histogram2d(x = zenit_rot, y = azimut_rot, bins = [bins,bins])
plt.imshow(imag_sph)
plt.show()

In [305]:
result_sph = hough_ellipse(imag_sph, accuracy = 1, threshold = 4)

In [306]:
# ordenar por acumulador creciente
result_sph.sort(order = 'accumulator')

#mejor acumulador: No muy relevante en este caso ya que no estamos identificando una elipse usando datos cartesianos
best = list(result_sph[-1])
best

[31, 27.0, 47.0, 20.248456731316587, 0.0, 2.788602265762883]

In [307]:
#extraer paramentos de elipse
yc, xc, a, b = (int(round(x)) for x in best[1:5]) #iterate over best[1:5]
orientation = best[5]

In [308]:
# extraer parametros de elipse
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
#cy, cx -> Index of pixels that belong to the ellipse perimeter

# Resaltar pixeles en la imagen original de modo a ver la elipse reconstruida
imag_sph[cy,cx] = 5

In [309]:
#plt.title('Branco: Anel reconstruido | Azul: dados')
plt.imshow(imag_sph, cmap = cm.nipy_spectral, aspect = 'auto')
plt.show()
#plt.savefig('figs/good recons/ellipse recons bins =' + str(bins) + '.jpg', format= 'jpg')

La aplicación directa de la transfomrda de Hough para una elipse en datos esfericos parece no resultar